# Food Delivery Optimization in Milan

## Introduction

In this project we will find the best placements for a number, in this case 10, of food couriers in Milan to optimize the delivery service with faster and better deliveries and also improve working conditions of couriers who will have to travel less distance.  

## Business Problem

In Milan there are many food delivery services and many more restaurants, so the optimal organization beetween these two fundamental components of the food business is essential to create an efficient and profitable system.
The target audience intersted in this project are the food delivery companies who work in the area of Milan because finding the optimal location in which the couriers will wait the next order is useful to reduce the time necessary to reach the restaurant, so the meal will be better, the client will wait less and the courier will travel less distance.

## Data Description

Using the Foursquare API we will retrieve the data of the restaurants and food services in the area of Milan, which has a location of 45°28'01" N and 09°11'24" E. For each activity we will retrieve name, position and type of cuisine. Through a KK-means algorithm with 10 point we will define the locations which minimize the distance beetween the couriers and the restaurants and finally we will analize the composition of each cluster of restaurants by their cousine.    

In [2]:
import numpy as np
import requests
import json

In [3]:
!conda install pandas==1.1.0 --yes
import pandas as pd
from pandas.io.json import json_normalize

Solving environment: ...working... done

# All requested packages already installed.



In [4]:
CLIENT_ID = 'TS4O02SW4CRHYCFOSATBLREBBH5QB3FIZD4IEDUPFZFUOLUV' # your Foursquare ID
CLIENT_SECRET = 'NGJ0VOEPCK4LCVQMXDUYRNSO2BO0ZW0OHQGV1TWRQQU4K4KY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TS4O02SW4CRHYCFOSATBLREBBH5QB3FIZD4IEDUPFZFUOLUV
CLIENT_SECRET:NGJ0VOEPCK4LCVQMXDUYRNSO2BO0ZW0OHQGV1TWRQQU4K4KY


In [5]:
milan_latitude = 45.4669444
milan_longitude = 9.19

In [6]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
categoryId = '4d4b7105d754a06374d81259' #Food category

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    milan_latitude, 
    milan_longitude, 
    radius, 
    LIMIT, 
    categoryId)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=TS4O02SW4CRHYCFOSATBLREBBH5QB3FIZD4IEDUPFZFUOLUV&client_secret=NGJ0VOEPCK4LCVQMXDUYRNSO2BO0ZW0OHQGV1TWRQQU4K4KY&v=20180605&ll=45.4669444,9.19&radius=500&limit=100&categoryId=4d4b7105d754a06374d81259'

In [7]:
results = requests.get(url).json()


In [8]:
# function that extracts the category of the venue
def get_category_type(row):

    categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
venues = json_normalize(results['response']['groups'][0]['items'])  

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues = venues.loc[:, filtered_columns]

categories=[]

for x in range(0, len(venues)):

    categories.append(get_category_type(venues.iloc[x]))
    
venues['venue.categories'] = categories
venues.columns = ['Name', 'Category', 'Latitude', 'Longitude']
print(venues)

                               Name            Category   Latitude  Longitude
0                             Luini              Bakery  45.465707   9.191431
1                          Spontini         Pizza Place  45.465655   9.191222
2   Ristorante Trussardi alla Scala  Italian Restaurant  45.467036   9.189512
3      Galleria Ristorante Pizzeria  Italian Restaurant  45.465881   9.190025
4                  VUN Andrea Aprea  Italian Restaurant  45.465290   9.189133
..                              ...                 ...        ...        ...
75                  Identità Golose          Restaurant  45.469006   9.190098
76           Ristorante Il Castello         Pizza Place  45.466658   9.184313
77                 Saluti da Modena              Bistro  45.469919   9.189443
78               Eccellenze Campane                Café  45.468769   9.185311
79                       Poke House          Poke Place  45.468478   9.185387

[80 rows x 4 columns]


C:\Users\danif\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.
